In [1]:
import io
import csv
import zipfile
import subprocess
from pathlib import Path
import pandas as pd
import requests
import shutil
from datetime import datetime
import os


In [2]:
download_dir = "../data_download"
os.makedirs(download_dir, exist_ok=True)

# Carpeta de salida para los Parquet
parquet_dir = "../data_parquet"
os.makedirs(parquet_dir, exist_ok=True)

fecha_inicio = '2020-01-01'
#fecha_fin = datetime.today()
fecha_fin = '2025-10-21'

In [3]:


# Descargamos el ZIP
url = "https://www.miteco.gob.es/content/dam/miteco/es/agua/temas/evaluacion-de-los-recursos-hidricos/boletin-hidrologico/Historico-de-embalses/BD-Embalses.zip"

print("Descargando BD-Embalses.zip desde MITECO...")
content = requests.get(url, timeout=120).content
zf = zipfile.ZipFile(io.BytesIO(content))

# Extrae el archivo .mdb a disco
mdb_members = [n for n in zf.namelist() if n.lower().endswith(".mdb")]
if not mdb_members:
    raise ValueError("No se encontró ningún archivo .mdb en el ZIP.")

download_dir = Path("../data_download")
download_dir.mkdir(exist_ok=True)
mdb_path = download_dir / Path(mdb_members[0]).name

with zf.open(mdb_members[0]) as f, open(mdb_path, "wb") as out:
    out.write(f.read())

print(f"Guardado en: {mdb_path}")

Descargando BD-Embalses.zip desde MITECO...
Guardado en: ../data_download/BD-Embalses.mdb


In [4]:
# Tabla que queremos extraer
table = "T_Datos Embalses 1988-2025"
mdb_path = f"{download_dir}/BD-Embalses.mdb"

# Verificamos que exista
tbls = subprocess.run(["mdb-tables", "-1", str(mdb_path)],
                      check=True, capture_output=True, text=True).stdout.strip().splitlines()

if table not in tbls:
    raise ValueError(f"La tabla '{table}' no está en el archivo MDB.")

# Exportamos la tabla a CSV y la cargamos con pandas
csv_bytes = subprocess.run(["mdb-export", str(mdb_path), table],
                           check=True, capture_output=True).stdout
df = pd.read_csv(io.BytesIO(csv_bytes), sep=",", quoting=csv.QUOTE_MINIMAL)


df["FECHA"] = pd.to_datetime(df["FECHA"], format="%d/%m/%y %H:%M:%S", errors="coerce")


# Filtramos desde 2020-01-01 en adelante
df_filtrado = df[(df["FECHA"] >= fecha_inicio) & (df["FECHA"] <= fecha_fin)].copy()
df_filtrado["year"] = df_filtrado["FECHA"].dt.year
df_filtrado[['AGUA_TOTAL','AGUA_ACTUAL']] = df_filtrado[['AGUA_TOTAL','AGUA_ACTUAL']].map(lambda x: float(x.replace(',', '.')) if isinstance(x, str) and ',' in x else x)
df_filtrado['porcentaje'] = df_filtrado['AGUA_ACTUAL'] / df_filtrado["AGUA_TOTAL"]

# Eliminamos si parquet_dir/embalses ya existe
if os.path.exists(f"{parquet_dir}/embalses"):
    shutil.rmtree(f"{parquet_dir}/embalses")

# Guardamos el DataFrame filtrado en Parquet
df_filtrado.to_parquet(f"{parquet_dir}/embalses", index=False, compression='snappy', partition_cols=['year'])

In [5]:
df_embalses = pd.read_parquet(f"{parquet_dir}/embalses")

df_embalses.info(), df_embalses.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42951 entries, 0 to 42950
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   AMBITO_NOMBRE   42951 non-null  object        
 1   EMBALSE_NOMBRE  42951 non-null  object        
 2   FECHA           42951 non-null  datetime64[ns]
 3   AGUA_TOTAL      42951 non-null  float64       
 4   AGUA_ACTUAL     42951 non-null  float64       
 5   ELECTRICO_FLAG  42951 non-null  int64         
 6   porcentaje      42951 non-null  float64       
 7   year            42951 non-null  category      
dtypes: category(1), datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 2.3+ MB


(None,
                       AMBITO_NOMBRE      EMBALSE_NOMBRE      FECHA  \
 34301  Cuenca Mediterránea Andaluza               Rules 2024-11-06   
 38394                         Duero  Porma-(Juan Benet) 2025-10-06   
 22917                    Miño - Sil       Santa Eulalia 2023-02-05   
 28341                          Ebro            Sallente 2023-03-01   
 40826                        Segura          La Pedrera 2025-01-04   
 
        AGUA_TOTAL  AGUA_ACTUAL  ELECTRICO_FLAG  porcentaje  year  
 34301       111.0         77.0               0    0.693694  2024  
 38394       318.0        301.0               0    0.946541  2025  
 22917        10.0          9.0               1    0.900000  2023  
 28341         6.0          2.0               1    0.333333  2023  
 40826       246.0         73.0               0    0.296748  2025  )